# TP GAN
## Prérequis

Avoir suivi la formation GAN et avoir quelques bases avec PyTorch. Vous pouvez vous mettre sous les yeux le TP CNN.

## Introduction

L'objectif de ce TP est de coder un GAN pour générer des petits chats trop mignons. Il abordera les notions principales vues pendant la formation GAN : les CNN, convolutions transposées, batchnorm, binary crossentropy, comment entraîner tout ce beau monde... (et peut-être les problèmes de GAN si ça arrive mais pas de panique).

La plupart des détails sont déjà implémentés, vous allez surtout être amené à travailler sur l'architecture du générateur et du discriminateur ainsi que de comprendre comment les entraîner.


## Les imports


On va d'abord importer les librairies nécessaires dont les classiques numpy, matplotlib ainsi que Pytorch (quand même....) et certains de ses modules et classes utilisés utilisés souvent (Dataset, nn)

In [ ]:
import torch
from torch.utils.data import Dataset
from torchvision import datasets
from torchvision.io import read_image
from torch.utils.data import DataLoader
from torch import nn

import matplotlib.pyplot as plt
from IPython.display import clear_output
import gdown
import os
from tqdm import tqdm
import pandas as pd

## Téléchargement du dataset


Importons maintenant le dataset. On va télécharget le dataset de chats depuis ce [drive](https://drive.google.com/uc?id=1F9I7iDmQ_I9Qsrav1UXlD4OiIBVSU5sl) avec la commande gdown (ou à la mano si vous préférez).


In [ ]:
#Téléchargement du dataset
url = 'https://drive.google.com/uc?id=1F9I7iDmQ_I9Qsrav1UXlD4OiIBVSU5sl'
output = 'dataset.tgz'
if not os.path.exists(output):
    gdown.download(url, output, quiet=False)

#Dézippage du dataset

def unzip(zip_file, dest_dir):
    import zipfile
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(dest_dir)

unzip('dataset.tgz', './')

#On répertorie les images dans un document csv

def create_csv(path):
    csv = open("images.csv", 'w')
    for file in os.listdir(path):
        csv.write(f"{file}\n")

    csv.close()

create_csv('./dataset')

On commence à traiter le dataset en répertoriant toutes ses images dans un document csv, dont on va pouvoir se servir pour itérer facilement sur nos données (chaque image à un indice).

## Quelques paramètres généraux


Les paramètres peuvent être modifiés pour tester un peu (sauf peut-être la taille de l'image pour ce dataset).

Pour ce dataset, on peut prendre une batch size pas trop grande sinon Colab crash (ou votre PC). De même, la dimension de l'espace latent peut être ajustée, ici vu que les chats c'est pas si simple, environ 100 c'est bien.

Pour charger le dataset, on crée une classe custom à partir de la classe 'Dataset', qui prend en entrée le chemin du dossier qui contient nos images (ici `'./dataset'`) ainsi que le document csv que nous avons crée juste avant (ici `'./images.csv'`). Les images ont leurs pixels entre 0 et 255 qu'on va renormaliser dès la création du dataset entre -1 et 1, ce qui est plus adapté pour les réseaux de neurones et bien pour des GAN car on a une moyenne nulle.

On crée également un dataloader à partir de notre dataset pour pouvoir itérer sur celui ci, ainsi que pour pouvoir batch et mélanger nos données.

On va aussi les afficher parce que c'est bien de savoir sur quoi on travaille quand même (et puis pourquoi se priver de photos de chats...).


In [ ]:
BATCH_SIZE = 64
LATENT_DIM = 100
IMG_SHAPE = (64,64,3)

#On crée un dataset custom à partir des images du dossier 'dataset'
class TrainDataset(Dataset):
    def __init__(self, img_csv, img_dir):
        self.img_dir = img_dir
        self.img_csv = pd.read_csv(img_csv)

    def __len__(self):
        return len(self.img_csv)

    def __getitem__(self, index):
        img_path = os.path.join(self.img_dir, self.img_csv.iloc[index, 0])
        image = read_image(img_path)
        #On normalise les données dans [-1, 1]
        image = ...
        return image

training_data = ...

#On charge notre dataset dans un dataloader
x_train = DataLoader(training_data, batch_size = BATCH_SIZE, shuffle = True)


In [ ]:
test_batch = next(iter(x_train))
test_batch = test_batch.permute(0,2,3,1)
fig = plt.figure(figsize=(12,12))
for i in range(25):
  plt.subplot(5,5,i+1)
  plt.axis('off')
  #On oublie pas de faire image * 0.5 + 0.5 pour revenir dans [0,1]
  plt.imshow(test_batch[i]*0.5+0.5)
plt.show()

## Pytorch et les GPUs...





Pytorch a besoin qu'on lui précise avec quel appareil on travaille (CPU ou GPU par exemple).

Du coup, ici on choisit le meilleur appareil disponible (généralement GPU, sinon CPU).

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

## Le discriminateur


On va d'abord faire le discriminateur.

Pour cela on va définir une sous-classe de nn.Module (on aurait pu faire simplement un nn.Sequential, mais on aura besoin des classes plus tard, de toute façon...)

En entrée, on va avoir une image de dimension (3,64,64) (c'est en couleur pour rappel donc il y a bien 3 canaux).

Pour vous aider, un petit rappel des blocs à mettre:
- `Conv2D(in_features,out_features,kernel_size,strides,padding)`

  Typiquement, on utilise une taille de kernel de 3 et des filtres qui sont des puissances de 2 (32,64,128...).
  
  Pour rappel, le padding `'same'` équivaut à rajouter des zéros pour garder la même taille d'image en sortie (si les strides sont de 1) et avec `'valid'` on en rajoute pas. `'same'` est donc conseillé ici pour éviter les surprises (et de se torturer la tête sur la taille en sortie même si vous savez évidemment que c'est $\frac{n+2p-f}{s}+1$). A savoir que sur Pytorch cela ne marche pas forcément, et que vous allez peut être devoir la préciser à la main.
   
   On ne met pas l'activation tout de suite car on ajoute d'abord de la batch normalization pour renormaliser le batch en 0 et ainsi profiter au maximum de la non-linéarité de la ReLU en ce point.

  Pour rappel, on augmente le nombre de filtres au fur et à mesure de l'architecture dans le discriminateur.
- `BatchNorm2D(in_features)`
- `LeakyReLU(alpha=0.2)` (pente de la leaky relu dans le domaine $]-\infty,0]$)

Mettez 3-4 blocs comme ça et faites des essais.

Ici pas de `Pooling` car on réduit la taille des images directement avec du stride (typiquement 2 à chaque Convolution en comptant bien la taille qu'on obtient à la fin).

En sortie on veut une dimension (1) grâce à une `Flatten` puis des `Dense` en oubliant pas la sigmoïde à la fin.

Si vous voulez pas trop vous embêter pour votre `forward`, vous pouvez utiliser un nn.Sequential dans votre modèle.

*Alternative* :
- C'est aussi possible de bien calculer la taille de l'image pour terminer par une `Convolution` avec une sortie de dimension (1,1,1) puis une `Flatten` (un peu mieux, dit "Full Convolutional").

  **Exemple** :
  - Image de (64,64,3) -> 4 Blocs de convolution avec du stride de 2 : $64/2^4$ -> Feature map de taille (4,4,nombre_de_filtres).

    On peut ensuite finir avec une `Conv2D` avec une taille de filtre de 4 et du padding `'valid'` pour juste faire une combinaison de tous les pixels restants, ce qui nous donne bien notre unique pixel de dimension (1,1,1) sans oublier la sigmoïde et on peut ensuite `Flatten` tout ça).


Juste ici, petite antisèche de l'architecture si vous séchez. Attention l'écriture est en pseudo-code, il faut l'adapter à Pytorch.
<details>
<summary>Antisèche</summary>
Conv2D(32 filtres,kernel taille 3,stride 2,padding 1) -> BN -> LR <br />  
-> Conv2D(64,kernel 3,stride 2,padding 1) -> BN -> LR <br />  
-> Conv2D(128,3,2,1) -> BN -> LR <br />  
-> Conv2D(256,3,2,1) -> BN -> LR <br />  
-> Conv2D(1,4,1,1) -> Sigmoïde -> Flatten ou Flatten -> Linear(1) -> Sigmoïde
</details>

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        # ici on définie les différentes couches de notre réseau
        ...

    def forward(self, x):
        # ici on définie comment une entrée est traitée par notre réseau
        x = ...
        return x

disc = Discriminator()

## Le générateur


On va faire le générateur.
En entrée : un vecteur de l'espace latent de dimension `latent_dim`.

`DensLinear(4 x 4 x 1024,input_shape=(latent_dim, ))`

Il n'y a pas de couches `Reshape` en Pytorch, donc on ne va pas pouvoir faire de `nn.Sequential` vu qu'il va nous falloir faire un `forward` un peu custom pour remedier à ça. On peut utiliser la fonction `torch.reshape(tensor, (-1, 1024, 4, 4))` par exemple.


On peut aussi directement redimensionner le vecteur de l'espace latent sans la couche Dense.


Pour vous aider, un petit rappel des blocs à mettre ensuite:
- `Conv2DTranspose(in_features,out_features,kernel_size,strides,padding,output_padding)`

  Pour rappel, on diminue le nombre de filtres au fur et à mesure de l'architecture dans le générateur (512,256,...)

- `BatchNorm2D(in_features)`
- `ReLU()`

Mettez 2-3 blocs comme ça et faites des essais aussi. **Faites bien attention** à la taille de vos images tout au long de l'architecture pour bien avoir la taille d'image finale voulue (n'hésitez pas à mettre des print de temps en temps pour vous aider à débuguer).

**Exemple**:

- On a transformé notre vecteur latent en image de dimension (1024,4,4). Pour avoir du (3,64,64), il faut donc 4 blocs de `Conv2DTranspose` (avec padding) pour avoir $4*2^4=64$. Libre à vous de changer l'entrée pour mettre le nombre de blocs que vous voulez.


Ici aussi pas de `UpSampling2D` car on augmente la taille avec du stride (2 aussi souvent) à chaque convolution transposée.

En sortie on veut une dimension (64,64,3) grâce à une `Conv2D` avec 3 filtres, on oublie pas de prendre une activation en tangente hyperbolique `'tanh'` pour avoir des pixels dans [-1,1].

<details>
<summary>Antisèche</summary>
  (Dans le doute on peut flatten ici)
  Dense(100,4*4*1024 neurones) -> Reshape(en (1024,4,4))
  -> ConvTransposée(256 filtres,kernel taille 3,stride 2,padding 1, output_padding 1) -> BN -> ReLU <br />  
  -> ConvTransposée(128,kernel 3,stride 2,padding 1, output_padding 1) -> BN -> ReLU <br />  
  -> ConvTransposée(64,3,2,1,1) -> BN -> ReLU <br />  
  -> ConvTransposée(32,3,2,1,1) -> BN -> ReLU <br />  
  -> ConvTransposée(3,3,1,1,1) -> Tanh
</details>

In [ ]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        # ici on définie les différentes couches de notre réseau
        ...

    def forward(self, x):
        # ici on définie comment une entrée est traitée par notre réseau
        x = ...
        return x

## Le train step


Cette partie est assez importante car elle permet de comprendre comment on entraîne vraiment un GAN, c'est-à-dire à quoi on compare les sorties pour entraîner correctement le discriminateur et le générateur. Ici, pas de `model.fit` malheureusement.
On va définir un `train_step`, c'est à dire ce qu'on va faire comme opérations à chaque batch :

Entraîner le discriminateur :

- Générer des images fausses à partir de bruit gaussien et en prédire les labels : on a besoin ici d'un vecteur latent de dimension `(batch_size,latent_dim,1,1)`. Ensuite, on fait passer ce bruit dans le générateur pour obtenir des fausses images. Enfin, on récupère la sortie du discriminateur sur celles-ci.


```
noise = torch.randn(shape).to(device) (merci pytorch...)
fake_images = generator(...)
fake_predictions = discriminator(...)
```


- Prendre des images vraies du dataset et en prédire aussi les labels. Donc la sortie du discriminateur sur les vraies images.


```
real_predictions = ...
```


- Calculer la loss en comparant les prédictions sur les fausses avec des 0 et les prédictions sur les vraies avec des 1.



La binary crossentropy prend en argument les labels visés puis ceux prédits.
Pour avoir des 1 ou des 0 : `torch.ones_like(tensor)` ou `torch.zeros_like(tensor)` avec `tensor` le tenseur dont on veut imiter la taille (par exemple les labels issus des prédictions, vu que c'est à ça que l'on va vouloir les comparer). Dans l'exemple suivant, cela corresponds aux `true_labels` et les prédictions précédentes aux `predictions`.
```
discriminator_loss_on_real = loss(real_labels,real_predictions)
discriminator_loss_on_fake = loss(fake_labels,fake_predictions)
discriminator_loss = discriminator_loss_on_real + discriminator_loss_on_fake
```

On peut éventuellement diviser la loss du discriminateur par 2.

- Calculer les gradients en fonction de la loss calculée et les différents paramètres du modèle
- On applique les gradients calculés avec l'optimisateur choisi

Entraîner le générateur (presque la même chose):
- Générer des images fausses à partir de bruit gaussien et en prédire les labels.
- Calculer la loss en comparant les prédictions sur les fausses avec des 1 (on veut tromper le discriminateur).
- Calculer les gradients en fonction de la loss calculée et les différents paramètres du modèle
- On applique les gradients calculés avec l'optimisateur choisi

In [ ]:
def train_step(real_images, generator, discriminator, loss, g_opt, d_opt):
  global LATENT_DIM
  batch_size = real_images.size(dim=0)


  noise = torch.randn(size = ...).to(device)
  fake_images = ...

  ### On entraine le discriminateur

  discriminator.zero_grad()

  fake_images_predictions = ...
  real_images_predictions = ...

  fake_labels = ...
  fake_labels = fake_labels.type_as(real_images)
  real_labels = ...
  real_labels = real_labels.type_as(real_images)

  ### On calcule la loss pour les vraies et les fausses images

  fake_loss = ...
  real_loss = ...

  disc_loss = (fake_loss + real_loss)/2

  ### Backpropagation

  disc_loss.backward()
  d_opt.step()
  #d_opt.zero_grad()



  ### On entraine le générateur

  generator.zero_grad()

  fake_images = ...
  fake_images_predictions = ...

  real_labels = ...
  real_labels = real_labels.type_as(real_images)

  ### On calcule la loss pour le générateur

  gen_loss = ...

  ### Backpropagation

  gen_loss.backward()
  g_opt.step()
  #g_opt.zero_grad()
  return gen_loss,disc_loss

## Le train

On code la fonction d'entraînement principale.
Il reste à compléter la loss et les optimisateurs à utiliser, typiquement ici la `BCELoss` et `Adam(parameters,lr=2e-4,betas = (0.5, 0.5)`.

Dans un premier temps, on peut prendre les mêmes optimiseurs pour les deux quitte à adapter pour tester après (changer le learning rate par exemple pour rééquilibrer un peu l'entraînement).

In [ ]:
def train(dataset,generator,discriminator,epochs,fixed_noise):

  ###A compléter###
  loss = ...
  g_opt = ...
  d_opt = ...

  Lgen_loss = []
  Ldisc_loss = []
  X = []
  j = 0

  for epoch in range(epochs):
    progress_bar = tqdm(dataset)
    ##Vu que c'est un dataloader, on ne peut itérer directement dessus avec son indice. On va juste prendre à chaque fois batch par batch.
    for _,image_batch in enumerate(progress_bar):
        j += 1
        real_images = image_batch.to(device)
        gen_loss, disc_loss = ...

        X.append(j)
        Lgen_loss.append(gen_loss.item())
        Ldisc_loss.append(disc_loss.item())

    clear_output(wait=False)
    generate_and_save_plots(X, Lgen_loss, Ldisc_loss) #Définie après, pour générer les courbes des loss
    summarize_performance(generator,fixed_noise) #Définie après, pour afficher les images générées

## L'affichage à chaque epoch


Petite fonction qui affiche les images obtenues à chaque epoch. On va **afficher** 25 images avec la même seed (toujours du même vecteur latent) pour voir l'amélioration progressive de l'image. Ce n'est pas de l'overfitting sur un seul vecteur car on **entraîne** bien à partir de vecteurs différents à chaque fois avant.

On affiche aussi après chaque epoch les courbes des loss du générateur et du discriminateur pour suivre l'entraînement et l'arrêter si on constate un souci.

In [ ]:
def summarize_performance(generator,fixed_noise):
  fake_images = generator(fixed_noise).detach().cpu()
  fake_images = torch.permute(fake_images, (0, 2, 3, 1))
  fig = plt.figure(figsize=(12,12))
  for i in range(25):
    plt.subplot(5,5,i+1)
    plt.axis('off')
    plt.imshow(fake_images[i]*0.5+0.5)
  plt.show()

In [ ]:
def generate_and_save_plots(X, Lgen_loss, Ldisc_loss):
    fig = plt.figure(figsize=(4,4))
    plt.plot(X,Lgen_loss, label = 'gen_loss')
    plt.plot(X,Ldisc_loss, label = 'disc_loss')
    plt.legend()
    plt.show()

## Ici vous lancez tout!

Choisissez le nombre d'epochs que vous voulez. Ici une vingtaine d'epochs peut suffire, on va pas attendre 3h quand même.

In [ ]:
generator = Generator(latent_dim=LATENT_DIM).to(device)
discriminator = Discriminator().to(device)
EPOCHS = ...
fixed_noise = torch.randn(size=...).to(device)
train(x_train,generator,discriminator,EPOCHS,fixed_noise)

## Sauvegarde du modèle et inférence

Maintenant que votre modèle s'est entraîné pendant 10 minutes voir plus, ce serait con de le perdre et de devoir le réentraîner à chaque fois (en plus ça pollue c'est pas cool).
On va donc voir comment le sauvegarder sur votre drive et ensuite l'inférer, c'est-à-dire générer les images sans toucher aux poids.

In [ ]:
#Mount son drive
from google.colab import drive
drive.mount('/content/drive')

# Google va vous demander alors de vous logger avec votre compte google et accepter les conditions blabla...

On peut sauvegarder uniquement les poids du modèle à chaque fois pour ne pas avoir à stocker d'autres informations inutiles.

In [ ]:
#On save les poids du générateur dans un fichier .h5 avec la méthode .save_weights
torch.save(generator,'/content/drive/My Drive/generator.h5')

#Faire pareil pour le discriminateur
...

Ensuite il ne reste plus qu'à load les poids des deux modèles, générer des vecteurs latents et inférer le modèle dessus.
Pour inférer le modèle, on a déjà fait ça dans la boucle d'entraînement, il suffit de le considérer comme une simple fonction et faire ```model(entree)```

In [ ]:
#On peut alors charger les poids d'un modèle déjà entraîné avec .load_weights
generator = torch.load('/content/drive/My Drive/generator.h5')

#Faire pareil pour le discriminateur
...

#On peut alors générer des images avec le générateur
latent_vector = torch.randn(size=(25, LATENT_DIM, 1, 1)).to(device) #Création de 25 vecteurs latents aléatoires pour générer 25 images
fake_images = generator(latent_vector).detach().cpu()               #Génération des images avec le générateur
fake_images = torch.permute(fake_images, (0, 2, 3, 1))              #On déplace les dimensions de notre tenseur pour les adapter à un format compatible avec plt

fig = plt.figure(figsize=(12,12))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.axis('off')
    plt.imshow(fake_images[i]*0.5+0.5)
plt.show()

# À développer

## Autres datasets

Vous pouvez essayer de générer des images sur d'autres datasets comme Fashion MNIST, CIFAR, CelebA ou n'importe quel type d'images qui vous font plaisir. S'il faut le télécharger et l'importer sur Colab, vous pouvez directement monter votre Drive et uploader votre dataset sur ce dernier.

## Autres architectures

Vous pouvez rajouter des blocs dans le générateur ou le discriminateur, essayer d'ajouter du Dropout, enlever les biais dans les couches, modifier la dimension de l'espace latent, initialiser les poids d'une certaine façon... Si vous êtes déjà à jour sur les CNN avancés (ResNET, MobileNet, EfficientNet), vous pouvez essayer de faire un GAN sur ces bases, ce qui permettra de résoudre certains problèmes comme les vanishing gradient.

## Tenter de forcer des problèmes

Faites n'importe quoi !

Plus sérieusement, cela peut arriver surtout sur des datasets plus compliqués, la plupart des choses fonctionnent pour MNIST à part si on fait exprès de provoquer des problèmes.

Vous pouvez essayer par exemple de déséquilibrer l'entraînement en modifiant les learning rate, en rajoutant beaucoup de couches seulement d'un côté...etc

Ici on peut forcer le mode collapse en prenant une dimension latente de seulement 1. Le générateur aura alors plus de risques de générer des images sur un seul mode.

## Tips d'entraînement possibles

- Utiliser une loss différente, du genre la Wasserstein loss.

- Changer le label visé pour les images vraies de 1 en 0.9

- Rajouter du bruit sur les images

- Ajouter des labels à l'entrée du générateur et du discriminateur (vous pourrez alors même choisir les classes des images générées)

- Entraîner le discrminateur plus que le générateur (typiquement entre 3 à 5 boucles d'entraînement à chaque fois que le générateur en fait une).


# Ce qu'il faut retenir de ce TP
- Architecture du générateur et du discriminateur :

  `Conv2D(Transpose) -> BatchNorm -> ReLU ou LeakyReLU`

  + Spécifités en entrée et sortie selon ce que l'on veut

  Eviter les fully connected/denses si possible

- Faire une boucle d'entraînement from scratch avec un train_step (très utile pour faire des choses plus compliquées)

  `Loss -> gradient_tape.gradient() -> optimizer.apply_gradients`